In [ ]:
from pygoridge import RPC, SocketRelay
import numpy as np
import json
from pygoridge.constants import PayloadType
import csv
from numpy import genfromtxt
import shap
my_data = genfromtxt('dataset_AI_Random_7.csv',delimiter=',', skip_header=1)

rpc = RPC(SocketRelay("127.0.0.1", 6001))

class memfile:
    def __init__(self):
        self.b = []
    def write(self, b):
        self.b += b
    def asbytes(self):
        return bytes(self.b)


def go_ur_ai_numpy(x):
    mm = memfile()
    np.save(mm, x)
    return np.array(json.loads(rpc("GoUr.InferNumpy", mm.asbytes(), PayloadType.PAYLOAD_RAW)))

with open('dataset_AI_Random_7.csv', 'r') as f:
    csv_reader = csv.reader(f)
    header = []
    for row in csv_reader:
        header = row
        break

#print(my_data)
print(go_ur_ai_numpy(my_data))

explainer = shap.KernelExplainer(go_ur_ai_numpy, my_data[:10], link="logit")
shap_values = explainer.shap_values(my_data[:100], nsamples=100)

In [ ]:
shap.initjs()
my_data_df = pd.DataFrame(my_data)
my_data_df.columns = header
shap.force_plot(explainer.expected_value, shap_values[1, :], my_data_df.iloc[1, :], link="logit")


In [ ]:
import pandas as pd
s = pd.Series(pd.DataFrame(shap_values).abs().mean())
df = pd.DataFrame(s)
df.index = header
df.sort_values(0, ascending=False)